In [1]:
import sys
sys.path.append("..")
import os, re, glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from src import data,unet,unet_dilation

Using TensorFlow backend.


In [2]:
from keras import utils
train_dir = "/home/ubuntu/TrainingSet"


save_imgs_dir = 'images/train250NoDrpAdam3/dilation_endo'

images = []
inner_masks = []
outer_masks = []

patient_directories = sorted(glob.glob(os.path.join(train_dir, "patient*")))

for patient_dir in patient_directories:
    imgdata = data.ImageData(patient_dir)
    images += imgdata.labeled_images
    inner_masks += imgdata.endo_masks
    outer_masks += imgdata.epi_masks

images = np.asarray(images)[:,:,:,None].astype('float64')
i_masks = np.asarray(inner_masks)
o_masks = np.asarray(outer_masks)

dims = i_masks.shape
classes = len(set(i_masks[0].flatten()))
new_shape = dims + (classes,)
i_masks = utils.to_categorical(i_masks).reshape(new_shape)
o_masks = utils.to_categorical(o_masks).reshape(new_shape)

#i_masks_cropped=[]
#crop_x = 8

#for mask in i_masks:
#    i_masks_cropped.append(mask[0+crop_x//2:216-crop_x//2,:,:])

#print(i_masks[0].shape)
#print(i_masks_cropped[0].shape)

#i_masks_final = np.asarray(i_masks_cropped)



def normalize(x, epsilon=1e-7, axis=(1,2)):
    x -= np.mean(x, axis=axis, keepdims=True)
    x /= np.std(x, axis=axis, keepdims=True) + epsilon
    
normalize(images,axis=(1,2))

print("There are %d total training images." % len(images))
print("There are %d total inner masks." % len(inner_masks))
print("There are %d total outer masks." % len(outer_masks))

There are 243 total training images.
There are 243 total inner masks.
There are 243 total outer masks.


In [3]:
height,width,_ = images[0].shape
dropout = 0.0
unet_conv = unet.UNet()

endo_model = unet_conv.get_unet(height=height, width=width, channels=1,features=32,steps=3,dropout=dropout,padding='same')
epi_model = unet_conv.get_unet(height=height, width=width, channels=1,features=32,steps=3,dropout=dropout,padding='same')

unet_dilate = unet_dilation.UNet()

endo_model_dilation = unet_dilate.get_unet(height=height,width=width,channels=1,features=32,steps=3,dropout=dropout,padding='same')
epi_model_dilation = unet_dilate.get_unet(height=height,width=width,channels=1,features=32,steps=3,dropout=dropout,padding='same')

In [4]:
from keras.optimizers import Adam, SGD
from keras import backend as K

def dice_coef(y_true, y_pred):
    flat_y_true = K.flatten(y_true)
    flat_y_pred = K.flatten(y_pred)
    intersection = K.sum(flat_y_true * flat_y_pred)
    return (2. * intersection + 1.) / (K.sum(flat_y_true) + K.sum(flat_y_pred) + 1.)

def dice_coef_np(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection + 1.) / (np.sum(y_true) + np.sum(y_pred) + 1.)

In [5]:
#generate test set from validation
import random
from keras.preprocessing.image import ImageDataGenerator

random.seed(78)

val_split = 0.2

split_index = int((1 - val_split) * len(images))

validation_images = images[split_index:]
validation_i_masks = i_masks[split_index:]
validation_o_masks = o_masks[split_index:]

val_batch_size = 1000
augmented_i_images = []
augmented_i_masks = []

augmented_o_images = []
augmented_o_masks = []
data_gen_args = dict(rotation_range=180,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     shear_range=0.1,
                     zoom_range=0.05,
                     fill_mode='nearest')

image_generator = ImageDataGenerator(**data_gen_args)
for val in range(val_batch_size):
    n = random.randint(0,len(validation_images)-1)
    i_image = validation_images[n]
    i_mask = validation_i_masks[n]
    o_image = validation_images[n]
    o_mask = validation_o_masks[n]
    
    _, _, channels = i_image.shape
    
    i_stacked = np.concatenate((i_image, i_mask), axis=2)
    o_stacked = np.concatenate((o_image, o_mask), axis=2)
    
    i_augmented = image_generator.random_transform(i_stacked)
    o_augmented = image_generator.random_transform(o_stacked)
    
    augmented_i_images.append(i_augmented[:,:,:channels])
    augmented_i_masks.append(np.round(i_augmented[:,:,channels:]))
    
    augmented_o_images.append(o_augmented[:,:,:channels])
    augmented_o_masks.append(np.round(o_augmented[:,:,channels:]))
    


print(len(augmented_i_images))
print(len(augmented_i_masks))
print(len(augmented_o_images))
print(len(augmented_o_masks))



1000
1000
1000
1000


In [6]:
import tensorflow as tf

weight_file_endo_dilation = 'notebooks/saved_models/endo_models/weights-250Epochs-NoDrop-Adam3-Dilation.hdf5'
weight_file_endo_base = 'notebooks/saved_models/endo_models/weights-500Epochs-NoDrop-Adam3.hdf5'
weight_file_epi_base='notebooks/saved_models/epi_models/weights-500Epochs-NoDrop-Adam3.hdf5'
weight_file_epi_dilation = 'notebooks/saved_models/epi_models/weights-250Epochs-NoDrop-Adam3-Dilation.hdf5'

endo_model.load_weights(weight_file_endo_base)
endo_model_dilation.load_weights(weight_file_endo_dilation)
epi_model.load_weights(weight_file_epi_base)
epi_model_dilation.load_weights(weight_file_epi_dilation)

def calculate_dice(images, masks_true,unet_model):
    dices = []
    masks_pred = np.concatenate([unet_model.predict(image[None,:,:,:]) for image in images])
    for mask_true, mask_pred in zip(masks_true, masks_pred):
        y_true = mask_true[:,:,1].astype('uint8')
        y_pred = np.round(mask_pred[:,:,1]).astype('uint8')
        dices.append(dice_coef_np(y_true.flatten(), y_pred.flatten()))
    print("Dice Average: {:.2f} Dice Stdev: {:.2f}".format(np.mean(dices), np.std(dices)))

print("Statistics for Regular UNet - Endocardium")
calculate_dice(augmented_i_images,augmented_i_masks,endo_model)
print("Statistics for Regular UNet - Epicardium")
calculate_dice(augmented_o_images,augmented_o_masks,epi_model)
print("Statistics for UNet with dilation rate - Endocardium")
calculate_dice(augmented_i_images,augmented_i_masks,endo_model_dilation)
print("Statistics for UNet with dilation rate - Epicardium")
calculate_dice(augmented_o_images, augmented_o_masks, epi_model_dilation)

Statistics for Regular UNet - Endocardium
Dice Average: 0.76 Dice Stdev: 0.28
Statistics for Regular UNet - Epicardium
Dice Average: 0.77 Dice Stdev: 0.28
Statistics for UNet with dilation rate - Endocardium
Dice Average: 0.76 Dice Stdev: 0.28
Statistics for UNet with dilation rate - Epicardium
Dice Average: 0.80 Dice Stdev: 0.26


In [7]:
validation_split = 0.1

split_index = int((1 - validation_split) * len(images))

train_images = images[:split_index]
train_masks = i_masks[:split_index]

val_images = images[split_index:]
val_masks = i_masks[split_index:]

print("Training dice for Regular UNet")
calculate_dice(train_images,train_masks,model)
print("Validation dice for Regular UNet")
calculate_dice(val_images,val_masks,model)

print("Training dice for UNet with Dropout")
calculate_dice(train_images,train_masks,model_drop)
print("Validation dice for UNet with Dropout")
calculate_dice(val_images,val_masks,model_drop)

print("Training dice for Regular UNet")
calculate_dice(train_images,train_masks,model_dilation)
print("Validation dice for Regular UNet")
calculate_dice(val_images,val_masks,model_dilation)

Training dice for Regular UNet
Dice Average: 0.93 Dice Stdev: 0.06
Validation dice for Regular UNet
Dice Average: 0.75 Dice Stdev: 0.30
Training dice for UNet with Dropout
Dice Average: 0.10 Dice Stdev: 0.06
Validation dice for UNet with Dropout
Dice Average: 0.07 Dice Stdev: 0.05
Training dice for Regular UNet
Dice Average: 0.90 Dice Stdev: 0.14
Validation dice for Regular UNet
Dice Average: 0.78 Dice Stdev: 0.24
